In [3]:
import cv2 as cv
import numpy as np
import random
import xml.etree.ElementTree as ET

MODE = 300
NUMBER = 8

No = []
place_x = []
place_y = 0
size=100

#画像、XMLデータの個数
DataSize=100

In [4]:
#画像、XMLの生成
def Make_PicXML(NAME):
    
    #サイズの倍率（60～100）
    size = random.randint(70,100)    
    x = int(MODE / NUMBER * size / 100)
    y = int(MODE / NUMBER * 19 / 14 * size / 100)
    
    #牌の種類の決定
    No=[]
    for num in range(NUMBER):
        No.append(random.randint(0,33))
    #print(No)
    
    #場所の決定
    place_x=[]
    place_x.append(random.randint( 0,int(MODE - x*NUMBER)))
    for num in range(NUMBER):
        place_x.append(place_x[0]+ x*(num+1))
    place_y=random.randint(1,int(MODE-y))

    #print(place_x)
    
    #画像の生成
    img=np.zeros((MODE,MODE,3))
    img=cv.rectangle(img,(0,0),(MODE,MODE),(0,200,50),cv.FILLED)

    for num in range(NUMBER):
        filename='sample\\'+str(No[num])+'.jpg'
        pai = cv.imread(filename)
        pai =cv.resize(pai,(x,y))
    
        img[place_y:place_y+y,place_x[num]:place_x[num]+x]=pai
    
    filename = './annotation/pic/' + str(NAME) + '.jpg'
    cv.imwrite(filename,img)
    
    #XMLファイルの生成
    Annotation = ET.Element('annotation')
    Filename = ET.SubElement(Annotation,'filename')
    Filename.text = str(NAME)+'.jpg'

    size = ET.SubElement(Annotation,'size')
    width = ET.SubElement(size,'width')
    width.text = str(MODE)
    height = ET.SubElement(size,'height')
    height.text = str(MODE)

    for num in range(NUMBER):
        Object = ET.SubElement(Annotation, 'object')
        name = ET.SubElement(Object, 'name')
        name.text =str(No[num]) 
        bndbox = ET.SubElement(Object, 'bndbox')
        xmin = ET.SubElement(bndbox, 'xmin')
        xmin.text = str(place_x[num])
        ymin = ET.SubElement(bndbox, 'ymin')
        ymin.text = str(place_y)
        xmax = ET.SubElement(bndbox, 'xmax')
        xmax.text = str(place_x[num] + x)
        ymax = ET.SubElement(bndbox, 'ymax')
        ymax.text = str(place_y + y)
    
    tree = ET.ElementTree(element=Annotation)
    filename = 'annotation\\xml\\' + str(NAME) +'.xml'
    tree.write(filename, encoding='utf-8', xml_declaration=True)


In [3]:
##ロバストをするとdimension？が変わってtrainingできなかった##

#画像のロバスト
def robust(KOSU):
    for num in range(KOSU):
        filename ='annotation\\pic\\' + str(num) + '.jpg'
    
        if num%5 == 3:
            #グレースケール化
            img=cv.imread(filename)
            gray=cv.cvtColor(img,cv.COLOR_RGB2GRAY)
            cv.imwrite(filename, gray)
    
        if num%5 == 4:
            #輝度変換
            gamma=2.0
            lookuptable =np.zeros((256,1),dtype='uint8')
            for i in range(256):
                lookuptable[i][0]=255*pow(float(i)/255 ,1.0/gamma)
            img=cv.imread(filename, 1)
            img_gamma = cv.LUT(img, lookuptable)
            cv.imwrite(filename, img_gamma)

In [5]:
for num in range(DataSize):
    Make_PicXML(num)